In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError, Accuracy
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Concatenate, Input, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l1_l2
from keras.models import Sequential
from transformers import TFAutoModel, AutoTokenizer


/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/clean_cards.csv')
df.sample(5)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
21295,2,['U'],False,['Instant'],['none'],Counter target spell unless its controller pay...,['Scry'],0,0,common,False,7240,True,0.05
21439,5,['W'],False,['Creature'],['Dragon'],"Flying\nWhenever an opponent casts a spell, pu...",['Flying'],4,3,rare,False,10733,False,0.70
17768,1,['U'],False,['Instant'],['none'],Tap all blue creatures.,['no keywords'],0,0,common,False,243,True,0.21
3431,7,['R'],False,['Creature'],"['Phyrexian', 'Giant']",You may reveal this card from your opening han...,['no keywords'],5,5,rare,False,6372,True,0.86
22017,2,['B'],False,['Creature'],"['Human', 'Rogue']",Temple Thief can't be blocked by enchanted cre...,['no keywords'],2,2,common,False,9550,True,0.02


In [3]:
no_prices_df = df[df['usd'].isna()]

In [4]:
df.drop(df[df['usd'].isna()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

#### Functions and models used for data processing

In [5]:
df['colors'] = df['colors'].apply(literal_eval)

In [6]:
mlb = MultiLabelBinarizer()
df= df.join(pd.DataFrame(mlb.fit_transform(df['colors']),
                                      columns=mlb.classes_,
                                      index=df.index))
legend_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['legendary']), columns=['legendary'])
             .add_prefix('_'))
booster_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['booster']), columns=['booster'])
             .add_prefix('_'))
reserved_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['reserved']), columns=['resrved'])
             .add_prefix('_'))

df = df.drop(['legendary', 'booster', 'reserved', 'colors'], axis=1)
df.head(3)

,cmc,types,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved
0,2,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0
1,4,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0
2,5,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0


#### Text processing

In [7]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = TFAutoModel.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [9]:
df['sub_types_embedding'] = None

for i in range(0, len(df), 32):
    batch_df = df.iloc[i:i+32]

    batch_tokens = tokenizer(list(batch_df['sub_types']), return_tensors='tf', padding=True, truncation=True)

    outputs = model(**batch_tokens)
    last_hidden_states = outputs.last_hidden_state

    batch_embeddings = tf.split(last_hidden_states, num_or_size_splits=len(batch_df))
    df.loc[i:i+32-1, 'sub_types_embedding'] = batch_embeddings

df['sub_types_embedding'] = df['sub_types_embedding'].tolist()


In [14]:
df['types_embedding'] = None

for i in range(0, len(df), 32):
    batch_df = df.iloc[i:i+32]

    batch_tokens = tokenizer(list(batch_df['types']), return_tensors='tf', padding=True, truncation=True)

    outputs = model(**batch_tokens)
    last_hidden_states = outputs.last_hidden_state

    batch_embeddings = tf.split(last_hidden_states, num_or_size_splits=len(batch_df))
    df.loc[i:i+32-1, 'types_embedding'] = batch_embeddings

df['types_embedding'] = df['types_embedding'].tolist()

In [15]:
df['keywords_embedding'] = None

for i in range(0, len(df), 32):
    batch_df = df.iloc[i:i+32]

    batch_tokens = tokenizer(list(batch_df['keywords']), return_tensors='tf', padding=True, truncation=True)

    outputs = model(**batch_tokens)
    last_hidden_states = outputs.last_hidden_state

    batch_embeddings = tf.split(last_hidden_states, num_or_size_splits=len(batch_df))
    df.loc[i:i+32-1, 'keywords_embedding'] = batch_embeddings

df['keywords_embedding'] = df['keywords_embedding'].tolist()

In [16]:
df['text_embedding'] = None

for i in range(0, len(df), 32):
    batch_df = df.iloc[i:i+32]

    batch_tokens = tokenizer(list(batch_df['oracle_text']), return_tensors='tf', padding=True, truncation=True)

    outputs = model(**batch_tokens)
    last_hidden_states = outputs.last_hidden_state

    batch_embeddings = tf.split(last_hidden_states, num_or_size_splits=len(batch_df))
    df.loc[i:i+32-1, 'text_embedding'] = batch_embeddings

df['text_embedding'] = df['text_embedding'].tolist()

In [ ]:
df.head(3)

,cmc,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved,sub_types_embedding,types_embedding,keywords_embedding,text_embedding
0,2,0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0,"[[0.048333920538425446, 0.09606622159481049, -...","[[-0.05527622252702713, 0.07099153101444244, -...","[[-0.039147913455963135, 0.0037941206246614456...","[[-0.36550939083099365, -0.362487256526947, -0..."
1,4,1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0,"[[-0.007577039301395416, -0.013978387229144573...","[[-0.004934143275022507, 0.021525928750634193,...","[[-0.011417698115110397, 0.05888311564922333, ...","[[-0.16684836149215698, -0.23930270969867706, ..."
2,5,5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0,"[[-0.05285697057843208, 0.10382992774248123, -...","[[-0.004934143275022507, 0.021525928750634193,...","[[0.08564460277557373, 0.04099284112453461, -0...","[[-0.23658572137355804, -0.27686968445777893, ..."


In [ ]:
df.drop(columns=['sub_types','types', 'keywords', 'oracle_text'],  inplace=True)

#### The Pipeline and neural network

In [ ]:
df

,cmc,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved,sub_types_embedding,types_embedding,keywords_embedding,text_embedding
0,2,0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0,"[[0.048333920538425446, 0.09606622159481049, -...","[[-0.05527622252702713, 0.07099153101444244, -...","[[-0.039147913455963135, 0.0037941206246614456...","[[-0.36550939083099365, -0.362487256526947, -0..."
1,4,1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0,"[[-0.007577039301395416, -0.013978387229144573...","[[-0.004934143275022507, 0.021525928750634193,...","[[-0.011417698115110397, 0.05888311564922333, ...","[[-0.16684836149215698, -0.23930270969867706, ..."
2,5,5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0,"[[-0.05285697057843208, 0.10382992774248123, -...","[[-0.004934143275022507, 0.021525928750634193,...","[[0.08564460277557373, 0.04099284112453461, -0...","[[-0.23658572137355804, -0.27686968445777893, ..."
3,0,0,0,common,2861,0.14,0,0,0,1,0,0,0,0,1,0,"[[-0.03981915861368179, -0.03859994560480118, ...","[[0.03863392025232315, -0.0012220675125718117,...","[[-0.0466962531208992, -0.020105689764022827, ...","[[-0.3683808147907257, -0.36620354652404785, -..."
4,3,0,0,rare,9634,0.09,0,1,0,0,0,0,0,0,0,0,"[[-0.03981915861368179, -0.03859994560480118, ...","[[-0.05527622252702713, 0.07099153101444244, -...","[[-0.0466962531208992, -0.020105689764022827, ...","[[-0.037254445254802704, -0.20066097378730774,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25484,5,7,2,mythic,7604,0.42,1,0,0,0,1,0,1,1,1,0,"[[-0.1253373622894287, 0.10831902921199799, -0...","[[-0.004934143275022507, 0.021525928750634193,...","[[-0.09200353920459747, 0.007305992767214775, ...","[[-0.2274055927991867, -0.26141682267189026, -..."
25485,4,0,0,rare,4258,0.82,0,0,0,0,0,1,0,0,1,0,"[[-0.03981915861368179, -0.03859994560480118, ...","[[0.00986480712890625, 0.047369033098220825, -...","[[-0.0466962531208992, -0.020105689764022827, ...","[[-0.33752167224884033, -0.2600155174732208, 0..."
25486,4,1,4,rare,10446,0.53,1,0,0,0,0,1,1,1,1,0,"[[-0.02251829020678997, 0.07062771171331406, -...","[[-0.004934143275022507, 0.021525928750634193,...","[[-0.011417698115110397, 0.05888311564922333, ...","[[-0.2219531089067459, -0.318474143743515, 0.0..."
25487,3,2,3,rare,10376,0.25,0,0,0,0,1,0,0,1,0,0,"[[-0.022037478163838387, 0.062315717339515686,...","[[-0.004934143275022507, 0.021525928750634193,...","[[-0.0466962531208992, -0.020105689764022827, ...","[[-0.3002524673938751, -0.3068389892578125, -0..."


In [ ]:
X = df.drop(columns='usd')
y = df['usd']

In [ ]:
numerical_features = ['power', 'toughness', 'released_at', 'cmc']
categorical_features = ['rarity']
remaining_features = X.drop(columns=['rarity', 'power', 'toughness', 'released_at', 'cmc'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(preprocessor)

In [ ]:
X_preprocessed= pipeline.fit_transform(X)

In [ ]:
column_names = pipeline.get_feature_names_out()
column_names

array(['num__power', 'num__toughness', 'num__released_at', 'num__cmc',
       'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare',
       'cat__rarity_special', 'cat__rarity_uncommon', 'remainder__B',
       'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R',
       'remainder__U', 'remainder__W', 'remainder___legendary',
       'remainder___booster', 'remainder___resrved',
       'remainder__sub_types_embedding', 'remainder__types_embedding',
       'remainder__keywords_embedding', 'remainder__text_embedding'],
      dtype=object)

In [ ]:
processed_df = pd.DataFrame(X_preprocessed, columns=column_names)

In [ ]:
processed_df.head(3)

,num__power,num__toughness,num__released_at,num__cmc,cat__rarity_common,cat__rarity_mythic,cat__rarity_rare,cat__rarity_special,cat__rarity_uncommon,remainder__B,...,remainder__R,remainder__U,remainder__W,remainder___legendary,remainder___booster,remainder___resrved,remainder__sub_types_embedding,remainder__types_embedding,remainder__keywords_embedding,remainder__text_embedding
0,-0.787786,-0.818064,0.83164,-0.74473,1.0,0.0,0.0,0.0,0.0,0,...,0,0,1,0,1,0,"[[0.048333920538425446, 0.09606622159481049, -...","[[-0.05527622252702713, 0.07099153101444244, -...","[[-0.039147913455963135, 0.0037941206246614456...","[[-0.36550939083099365, -0.362487256526947, -0..."
1,-0.237532,1.302998,0.930864,0.397826,1.0,0.0,0.0,0.0,0.0,0,...,0,1,0,0,1,0,"[[-0.007577039301395416, -0.013978387229144573...","[[-0.004934143275022507, 0.021525928750634193,...","[[-0.011417698115110397, 0.05888311564922333, ...","[[-0.16684836149215698, -0.23930270969867706, ..."
2,1.963486,1.833263,0.967534,0.969104,0.0,1.0,0.0,0.0,0.0,1,...,1,1,0,1,0,0,"[[-0.05285697057843208, 0.10382992774248123, -...","[[-0.004934143275022507, 0.021525928750634193,...","[[0.08564460277557373, 0.04099284112453461, -0...","[[-0.23658572137355804, -0.27686968445777893, ..."


In [ ]:
processed_df =processed_df.astype({
    'num__power': float,
    'num__toughness': float,
    'num__released_at': float,
    'num__cmc': float
})

In [ ]:
processed_df = processed_df.astype({
    "cat__rarity_common": int,
    "cat__rarity_mythic": int,
    "cat__rarity_rare": int,
    "cat__rarity_special": int,
    "cat__rarity_uncommon": int,
    "remainder__B": int,
    "remainder__C": int,
    "remainder__G": int,
    "remainder__N": int,
    "remainder__R": int,
    "remainder__U": int,
    "remainder__W": int,
    "remainder___legendary": int,
    "remainder___booster": int,
    "remainder___resrved": int
})

In [ ]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25489 entries, 0 to 25488
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   num__power                      25489 non-null  float64
 1   num__toughness                  25489 non-null  float64
 2   num__released_at                25489 non-null  float64
 3   num__cmc                        25489 non-null  float64
 4   cat__rarity_common              25489 non-null  int64  
 5   cat__rarity_mythic              25489 non-null  int64  
 6   cat__rarity_rare                25489 non-null  int64  
 7   cat__rarity_special             25489 non-null  int64  
 8   cat__rarity_uncommon            25489 non-null  int64  
 9   remainder__B                    25489 non-null  int64  
 10  remainder__C                    25489 non-null  int64  
 11  remainder__G                    25489 non-null  int64  
 12  remainder__N                    

In [ ]:
X_numerical = processed_df[['num__power', 'num__toughness', 'num__released_at', 'num__cmc']]
X_categorical = processed_df[['remainder___legendary', 'remainder___booster', 'remainder___resrved',
                              'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare', 'cat__rarity_special', 'cat__rarity_uncommon',
                              'remainder__B', 'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R', 'remainder__U', 'remainder__W']]
X_text = processed_df[['remainder__text_embedding']]
X_subtypes= processed_df[['remainder__sub_types_embedding']]
X_types = processed_df[['remainder__types_embedding']]
X_keys = processed_df[['remainder__keywords_embedding']]

In [ ]:
X_numerical.shape, X_categorical.shape, X_text.shape, X_subtypes.shape, X_types.shape, X_keys.shape

((25489, 4), (25489, 15), (25489, 1), (25489, 1), (25489, 1), (25489, 1))

In [ ]:
X_text['remainder__text_embedding']=X_text['remainder__text_embedding'].apply(np.array)
X_subtypes['remainder__sub_types_embedding']=X_subtypes['remainder__sub_types_embedding'].apply(np.array)
X_types['remainder__types_embedding']=X_types['remainder__types_embedding'].apply(np.array)
X_keys['remainder__keywords_embedding']=X_keys['remainder__keywords_embedding'].apply(np.array)

/var/folders/tm/jmhp8xvj4_d4hh_01cf1jt_r0000gn/T/ipykernel_26572/1449457955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_text['remainder__text_embedding']=X_text['remainder__text_embedding'].apply(np.array)
/var/folders/tm/jmhp8xvj4_d4hh_01cf1jt_r0000gn/T/ipykernel_26572/1449457955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_subtypes['remainder__sub_types_embedding']=X_subtypes['remainder__sub_types_embedding'].apply(np.array)
/var/folders/tm/jmhp8xvj4_d4hh_01cf1jt_r0000gn/T/ipykernel_265

In [ ]:
np.unique(X_text['remainder__text_embedding'].apply(lambda x: x.shape)), np.unique(X_subtypes['remainder__sub_types_embedding'].apply(lambda x: x.shape)), np.unique(X_types['remainder__types_embedding'].apply(lambda x: x.shape)), np.unique(X_keys['remainder__keywords_embedding'].apply(lambda x: x.shape))


(array([(1, 768)], dtype=object),
 array([(1, 768)], dtype=object),
 array([(1, 768)], dtype=object),
 array([(1, 768)], dtype=object))

In [ ]:
X_text_embeddings = np.concatenate(X_text['remainder__text_embedding'].values).reshape(-1, 768)
X_subtypes_embedding = np.concatenate(X_subtypes['remainder__sub_types_embedding'].values).reshape(-1, 768)
X_types_embedding = np.concatenate(X_types['remainder__types_embedding'].values).reshape(-1, 768)
X_keys_embedding = np.concatenate(X_keys['remainder__keywords_embedding'].values).reshape(-1, 768)

In [ ]:
X_text_embeddings[0].shape

(768,)

In [ ]:
X_numerical_train, X_numerical_test, X_categorical_train, X_categorical_test, X_text_train, X_text_test,X_subtypes_train, X_subtypes_test, X_types_train, X_types_test, X_keys_train, X_keys_test, y_train, y_test = train_test_split(
    X_numerical, X_categorical, X_text_embeddings, X_subtypes_embedding, X_types_embedding, X_keys_embedding, y, test_size=0.30, random_state=42
)

In [ ]:
def initialize_model():

    model_numerical = Sequential([
        Input(shape=(4,), name='input_numerical'),
        Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu')
    ], name='model_numerical')

    model_categorical = Sequential([
        Input(shape=(15,), name='input_categorical'),
        Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu')
    ], name='model_categorical')

    # Create a Sequential model for the text input
    model_text = Sequential([
        Input(shape=(768,), name='input_text'),
        Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
    ], name='model_text')

    model_type = Sequential([
        Input(shape=(768,), name='input_type'),
        Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
    ], name='model_type')


    model_subtype = Sequential([
        Input(shape=(768,), name='input_subtype'),
        Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_subtype')

    model_keys = Sequential([
        Input(shape=(768,), name='input_keys'),
        Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_keys')

    # Concatenate the outputs of all Sequential models
    concatenated = Concatenate()([
        model_numerical.output,
        model_categorical.output,
        model_text.output,
        model_type.output,
        model_keys.output,
        model_subtype.output
    ])

    output = Dense(1, activation='linear', name='output')(concatenated)


    model = models.Model(inputs=[model_numerical.input, model_categorical.input, model_text.input, model_type.input, model_keys.input, model_subtype.input], outputs=output)

    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['mse', 'accuracy'])

    return model


In [ ]:
def get_history():
    es = EarlyStopping(patience=10, restore_best_weights=False)

    train_data = {
        'input_numerical': X_numerical_train,
        'input_categorical': X_categorical_train,
        'input_text': X_text_train,
        'input_type': X_types_train,
        'input_subtype': X_subtypes_train,
        'input_keys': X_keys_train
    }

    test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }

    history = model.fit(
        train_data,
        y_train,
        epochs=20,
        batch_size=16,
        validation_data=(test_data, y_test),
        callbacks=[es],
        verbose=1
    )

    return history


In [ ]:
model = initialize_model()

In [ ]:
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_text (InputLayer)     [(None, 768)]                0         []                            
                                                                                                  
 dense_120 (Dense)           (None, 128)                  98432     ['input_text[0][0]']          
                                                                                                  
 dense_121 (Dense)           (None, 128)                  16512     ['dense_120[0][0]']           
                                                                                                  
 dropout_13 (Dropout)        (None, 128)                  0         ['dense_121[0][0]']           
                                                                                           

In [ ]:
get_history()

Epoch 1/20
1116/1116 [==============================] - 9s 6ms/step - loss: 1870.1902 - mse: 1856.6372 - accuracy: 8.4071e-04 - val_loss: 2004.7373 - val_mse: 2000.9058 - val_accuracy: 2.6154e-04
Epoch 2/20
1116/1116 [==============================] - 7s 6ms/step - loss: 1835.0253 - mse: 1830.7142 - accuracy: 7.2862e-04 - val_loss: 1964.5713 - val_mse: 1960.2598 - val_accuracy: 2.6154e-04
Epoch 3/20
1116/1116 [==============================] - 7s 6ms/step - loss: 1805.4146 - mse: 1801.5183 - accuracy: 6.1652e-04 - val_loss: 1949.7584 - val_mse: 1946.9307 - val_accuracy: 2.6154e-04
Epoch 4/20
1116/1116 [==============================] - 7s 7ms/step - loss: 1802.7318 - mse: 1799.4714 - accuracy: 7.2862e-04 - val_loss: 1925.6992 - val_mse: 1923.0122 - val_accuracy: 2.6154e-04
Epoch 5/20
1116/1116 [==============================] - 8s 7ms/step - loss: 1778.4688 - mse: 1775.6964 - accuracy: 7.8467e-04 - val_loss: 1907.5577 - val_mse: 1903.3196 - val_accuracy: 1.3077e-04
Epoch 6/20
1116/1116

In [ ]:
test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }
model.evaluate(test_data, y_test)

239/239 [==============================] - 0s 2ms/step - loss: 1885.5562 - mse: 1881.6256 - accuracy: 2.6154e-04


[1885.55615234375, 1881.6256103515625, 0.00026154046645388007]

In [ ]:
no_prices_df = df[df['usd'].isna()]